In [5]:
import sympy as sp
import tempfile

# Function to calculate Christoffel symbols for any metric
def christoffel_symbols(metric, coords):
    n = len(coords)  # Dimension
    
    # inverse of the metric
    g_inv = metric.inv()

    # Initialize the Christoffel symbols as a 3D tensor (n x n x n)
    Gamma = [[[0 for _ in range(n)] for _ in range(n)] for _ in range(n)]

    # Compute the Christoffel symbols
    for i in range(n):
        for j in range(n):
            for k in range(n):
                # Christoffel symbol \Gamma^k_{ij}
                Gamma[i][j][k] = 0
                for l in range(n):
                    Gamma[i][j][k] += 1/2 * g_inv[k, l] * (
                        sp.diff(metric[l, j], coords[i]) +
                        sp.diff(metric[l, i], coords[j]) -
                        sp.diff(metric[j, i], coords[l])
                    )
                Gamma[i][j][k] = sp.simplify(Gamma[i][j][k])
    
    return Gamma

def codazzi_mainardi(Gamma, A, coords):
    """
    Function to compute the Codazzi-Mainardi equations in terms of the second fundamental form A.
    
    :param Gamma: A list containing the Christoffel symbols Γ^k_ik
    :param A: A dictionary containing the second fundamental form components A_ij.
              E.g., A[i, j] = A_ij.
    :param coords: List of coordinate symbols [x1, x2, ..., xn] for the surface.
    :return: Codazzi-Mainardi equations in terms of symbolic expressions.
    """
    # Note that we did not use the symmetry of the second fundamental form one has to account for it when the applying it to a specific example. See below

    n = len(coords)  # Dimension of the manifold
    eqs = []  # List for Codazzi-Mainardi equations

    for i in range(n):
        for j in range(n):
            for k in range(n):
                #Derivative with respect to i of A[k,j]
                partial_A_kj_i = sp.diff(A[(k, j)], coords[i])
                #Derivative with respect to j of A[i,k]
                partial_A_ik_j = sp.diff(A[(i, k)], coords[j])

                # Codazzi-Mainardi equation for the current indices
                codazzi_eq = sp.simplify(partial_A_kj_i - partial_A_ik_j +
                                         sum(Gamma[k][j][r] * A[(r, i)] - Gamma[k][i][r] * A[(r, j)] for r in range(n)))
                eqs.append(((i+1, j+1, k+1), codazzi_eq))
    
    return eqs


# Example: Conformal metric i.e. g_{ij} = e^{2\lambda} * delta_{ij}
n = 2
coords = sp.symbols('x1 x2')  

# Define the conformal factor \lambda(x1, x2)
_lambda = sp.Function('lambda')(coords[0], coords[1])

metric = sp.exp(2 * _lambda) * sp.eye(n)

# Define the second fundamental form A_ij
A = {
    (0, 0): sp.Function('A_00')(coords[0], coords[1]),
    (0, 1): sp.Function('A_01')(coords[0], coords[1]),
    (1, 0): sp.Function('A_01')(coords[0], coords[1]),  # Symmetry: A_10 = A_01
    (1, 1): sp.Function('A_11')(coords[0], coords[1])
}

# Compute the Christoffel symbols
Gamma = christoffel_symbols(metric, coords)

#Compzre the Codazzi-Mainardi equations

codazzi_equations = codazzi_mainardi(Gamma, A, coords)

# Save the results in a temporary file
def save_results_to_temp_file(Gamma, codazzi_equations):
    with tempfile.NamedTemporaryFile(delete=False, suffix='.txt', mode='w') as f:
        temp_filename = f.name
        # Write the Christoffel symbols
        f.write("Christoffel Symbols:\n")
        for i in range(2):
            for j in range(2):
                for k in range(2):
                    if Gamma[i][j][k] != 0:
                        f.write(f"Gamma^{k+1}_{i+1}{j+1} = {sp.simplify(Gamma[i][j][k])}\n")

        # Write the Codazzi-Mainardi equations
        f.write("\nCodazzi-Mainardi Equations:\n")
        for indices, eq in codazzi_equations:
            if eq != 0:
                f.write(f"Codazzi equation for indices ({indices[0]}, {indices[1]}, {indices[2]}):\n")
                f.write(str(sp.Eq(0, eq)) + "\n")

    return temp_filename

# Save the results to a temporary file
temp_filename = save_results_to_temp_file(Gamma, codazzi_equations)

print(f"Results saved to temporary file: {temp_filename}")


Results saved to temporary file: /var/folders/v9/_5syljlj46x7sjwvgknn5gmr0000gn/T/tmpjnpy_v4w.txt
